<a href="https://colab.research.google.com/github/apploberry/sa-competition/blob/main/EN-SA/EN-SA%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import json

In [3]:
#### Constraint 설정 ####

# 1. Competition 제출 모드 
#    True : Competition 제출 파일 생성
#    False : test dataset 검증
exam_mode = True

# 2. 대화 길이
sentence_len = 4

# 3. Sentence에 Speaker 추가 
flag_speaker = True

# 4. MAX_LEN
#    0으로 설정하면 Tokenized Text의 최대길이로 설정
MAX_LEN = 0

# 5. Pre-trained Model Name
PRETRAINED_MODEL = 'bert-base-multilingual-cased'

# 6. Batch Size
BATCH_SIZE = 24

# 7. 에폭수
EPOCHS = 4

# 8. RANDOM SEED
SEED = 42


In [4]:
#### 학습 Data를 가져오기 ####

!git clone https://github.com/apploberry/sa-competition.git


with open('sa-competition/EN-SA/data/Friends_train.json', 'r', encoding='UTF-8') as f:
    json_train = json.load(f)

train = pd.array(json_train)
print('training set: ', train.shape)

with open('sa-competition/EN-SA/data/Friends_dev.json', 'r', encoding='UTF-8') as f:
    json_dev = json.load(f)

dev = pd.array(json_dev)
print('dev set: ', dev.shape)

fatal: destination path 'sa-competition' already exists and is not an empty directory.
training set:  (720,)
dev set:  (80,)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [5]:
#### Test Data load ####
test = []
sen_set = []
i = 0

if exam_mode:  # Competition 실습인 경우
    with open('sa-competition/EN-SA/data/en_data.csv', 'r', encoding='utf-8') as f:
        data = pd.read_csv(f)

    for j in range(len(data)):
        if(data['i_dialog'][j] != i):
            i = data['i_dialog'][j]
            test.append(sen_set)
            sen_set = []

        sen = {'speaker':data['speaker'][j], 'utterance':data['utterance'][j], 'emotion': 'neutral'}  # emotion은 임의로 넣어줌
        sen_set.append(sen)
    test.append(sen_set)
else:  # test인 경우
    with open('sa-competition/EN-SA/data/Friends_test.json', 'r', encoding='UTF-8') as f:
        test = json.load(f)

test = pd.array(test)
print('test set: ', test.shape)

test set:  (151,)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [6]:
#### Function 선언 ####

def labeltoint(lbl):
    return {'neutral': 0, 'non-neutral': 1, 'joy': 2, 'sadness': 3, 'anger': 4, 'surprise': 5, 'fear': 6, 'disgust': 7}[lbl]

def inttolabel(lbl):
    return {0: 'neutral', 1: 'non-neutral',2: 'joy', 3: 'sadness', 4: 'anger', 5: 'surprise', 6: 'fear', 7: 'disgust'}[lbl]

def speakertoint(lbl):
    return {'Chandler': 0, 'Joey': 1, 'Ross': 2, 'Rachel': 3, 'Monica': 4, 'Phoebe': 5, 'Other': 6}[lbl]

def inttospeaker(lbl):
    return {0: 'Chandler', 1: 'Joey',2: 'Ross', 3: 'Rachel', 4: 'Monica', 5: 'Phoebe', 6:'Other'}[lbl]

def speakerCheck(lbl):
    if lbl == 'Chandler' or lbl == 'Joey' or lbl == 'Ross' or lbl == 'Rachel' or lbl == 'Monica' or lbl == 'Phoebe':
        return lbl
    else:
       return 'Other'


# encoding 처리 (임시)
def replaceunicode(text):
    text = text.replace('\u0085', " ")
    text = text.replace('\u0091', "'")
    text = text.replace('\u0092', "'")
    text = text.replace('\u0093', "'")
    text = text.replace('\u0094', "'")
    text = text.replace('\u0097', ", ")
    text = text.replace('\u2019', "'")
    text = text.replace('\u00a0', "")
    text = text.replace('\u2026', "...")
    text = text.replace('\u2014', ". ")
    text = text.replace('\u00e9', "")
    text = text.replace('\u0096', ".")
    text = text.replace('\u00e8', "!")
    return text


# BERT 입력 형식에 맞게 문장 구성 (대화 길이에 맞춰서 구성)
def Make_Sentence(df, speaker, sentence_len):

    empty_str = ""
    result = ""

    empty_len = sentence_len - len(df)

    for s, p in zip(df, speaker):

        if flag_speaker:  # Speaker를 사용할 경우 메인 player 6명은 앞에 이름을 붙여줌
            if p in {'Chandler', 'Joey', 'Ross', 'Rachel', 'Monica', 'Phoebe'}:
              result =  result + ' {0}'.format(p)

        result = result + " " + replaceunicode(s) + " [SEP]"

    result = empty_str + result
    result = "[CLS]" + result

    return result


# 입력받은 대화를 학습용 문장으로 변경
def makeTalkingSet(trainingSet):
    sentences = []
    labels = []
    speaker = []
    for talking_set in trainingSet:
        df_talk = pd.DataFrame(talking_set)
        
        for i in range(df_talk.index.size):
            sentence = df_talk.iloc[max(0,i-sentence_len+1):i+1]['utterance']
            sentence_speaker = df_talk.iloc[max(0,i-sentence_len+1):i+1]['speaker']
            dummy_sen = Make_Sentence(sentence, sentence_speaker, sentence_len)
            sentences.append(dummy_sen)
            labels.append(labeltoint(df_talk.iloc[i]['emotion']))
            speaker.append(speakertoint(speakerCheck(df_talk.iloc[i]['speaker'])))
    
    sentences = pd.Series(sentences)

    return sentences, labels, speaker


In [7]:
#### train, dev(검증), test 데이터의 문장을 BERT 학습 형태로 수정 ####

sentences_train, labels_train, speaker_train = makeTalkingSet(train)
sentences_dev, labels_dev, speaker_dev = makeTalkingSet(dev)
sentences_test, labels_test, speaker_test = makeTalkingSet(test)

print(sentences_train.shape)
print(sentences_train[:10])
print(len(labels_train))
print(labels_train[:100])

(10561,)
0    [CLS] Chandler also I was the point person on ...
1    [CLS] Chandler also I was the point person on ...
2    [CLS] Chandler also I was the point person on ...
3    [CLS] Chandler also I was the point person on ...
4    [CLS] You must've had your hands full. [SEP] C...
5    [CLS] Chandler That I did. That I did. [SEP] S...
6    [CLS] So let's talk a little bit about your du...
7    [CLS] Chandler My duties?  All right. [SEP] No...
8    [CLS] Now you'll be heading a whole division, ...
9    [CLS] Chandler I see. [SEP] But there'll be pe...
dtype: object
10561
[0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 6, 0, 5, 0, 5, 1, 5, 1, 0, 0, 1, 1, 0, 0, 0, 2, 3, 5, 0, 1, 3, 0, 0, 2, 0, 1, 5, 5, 5, 0, 0, 0, 5, 1, 3, 0, 1, 1, 5, 0, 0, 0, 0, 0, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 5, 2, 5, 2, 0, 0, 1, 1, 0, 1, 4, 4, 1, 0, 0, 3, 3, 3, 5, 1, 1, 4, 1, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2]


In [8]:
#### BERT Tokenizer 로 데이터 Tokenize ####

tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL, do_lower_case=False)
tokenized_texts_train = [tokenizer.tokenize(sent) for sent in sentences_train]
tokenized_texts_test = [tokenizer.tokenize(sent) for sent in sentences_test]
tokenized_texts_dev = [tokenizer.tokenize(sent) for sent in sentences_dev]

print(sentences_train[0])
print(tokenized_texts_train[0])

[CLS] Chandler also I was the point person on my company's transition from the KL-5 to GR-6 system. [SEP]
['[CLS]', 'Chandler', 'also', 'I', 'was', 'the', 'point', 'person', 'on', 'my', 'company', "'", 's', 'transition', 'from', 'the', 'K', '##L', '-', '5', 'to', 'G', '##R', '-', '6', 'system', '.', '[SEP]']


In [11]:
#### Tokenized Test 최대 길이를 구해서 MAX_LEN 설정 ####

# constraint 에서 MAX_LEN = 0 인 경우에만 구함

if MAX_LEN == 0:
    str_len = []
    for s in tokenized_texts_train:
        str_len.append(len(s))
    for s in tokenized_texts_test:
        str_len.append(len(s))
    for s in tokenized_texts_dev:
        str_len.append(len(s))
        
    MAX_LEN = max(str_len)
    print('tokenized text의 최대 길이 : ',MAX_LEN)


In [12]:
#### tokens to ids 수행 ####

input_ids_train = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_train]
input_ids_train = pad_sequences(input_ids_train, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

input_ids_test = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test]
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

input_ids_dev = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_dev]
input_ids_dev = pad_sequences(input_ids_dev, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

input_ids_train[0]

array([  101, 14394,  1145,   146,  1108,  1103,  1553,  1825,  1113,
        1139,  1419,   112,   188,  6468,  1121,  1103,   148,  2162,
         118,   126,  1106,   144,  2069,   118,   127,  1449,   119,
         102,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [13]:
#### Attention Mask 생성 ####

def set_attention_masks(input_ids):
    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
      
    return attention_masks

attention_masks_train = set_attention_masks(input_ids_train)
attention_masks_test = set_attention_masks(input_ids_test)
attention_masks_dev = set_attention_masks(input_ids_dev)

print(attention_masks_train[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [14]:
#### tensor 로 변환 ####

train_inputs = torch.tensor(input_ids_train)
train_labels = torch.tensor(labels_train)
train_masks = torch.tensor(attention_masks_train)
validation_inputs = torch.tensor(input_ids_dev)
validation_labels = torch.tensor(labels_dev)
validation_masks = torch.tensor(attention_masks_dev)

print(train_inputs.size())
print(train_labels.size())
print(train_masks.size())
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])


torch.Size([10561, 155])
torch.Size([10561])
torch.Size([10561, 155])
tensor([  101, 19704,  2048,  1139,  1875,   117,  1119,   112,   188,  1575,
         1122,   119,  1124,   112,   188,  5733,  1575,  1122,   119,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,  

In [15]:
#### train_dataloader 준비 ####

batch_size = BATCH_SIZE

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [16]:
#### test_dataloader 준비 ####

# test data는 sample 추출시 SequentialSampler 사용

test_inputs = torch.tensor(input_ids_test)
test_labels = torch.tensor(labels_test)
test_masks = torch.tensor(attention_masks_test)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [17]:
#### GPU Name 확인 ####

device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [18]:
#### Cuda GPU 환경 확인 ####

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [19]:
#### Pre-trained Model 불러오기 ####

model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=8)
model.cuda()

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1

In [20]:
#### 학습 환경 설정 ####

# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-6, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = EPOCHS

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [21]:
#### 학습 관련 함수 선언 ####

# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))

    # hh:mm:ss으로 형태 변경
    return datetime.timedelta(seconds=elapsed_rounded)

In [23]:
#### 모델 학습 ####

# 랜덤시드 고정
seed_val = SEED

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행      
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 1 ========
Training...
  Batch   100  of    441.    Elapsed: 0:01:10.
  Batch   200  of    441.    Elapsed: 0:02:19.
  Batch   300  of    441.    Elapsed: 0:03:29.
  Batch   400  of    441.    Elapsed: 0:04:39.

  Average training loss: 1.56
  Training epcoh took: 0:05:07

Running Validation...
  Accuracy: 0.49
  Validation took: 0:00:11

Training complete!


In [24]:
#### 모델 TEST ####

#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

test_logits = []
test_label_ids = []

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for logit in logits:
        test_logits.append(logit)

    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1


# numpy로 변환
test_logits = np.array(test_logits)
test_label_ids = np.array(test_label_ids)

# 결과 보여줌
print("")
if exam_mode == False: # 정확도는 exam_mode = False 일때만 보여준다. Competition data는 emotion을 임의로 넣어서 의미 없으므로 보여주지 않음
    print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch    50  of     68.    Elapsed: 0:00:11.

Test took: 0:00:15


In [34]:
#### Competition 결과 저장 ####

import csv

if exam_mode == True: # competition data 일때만 수행

    result = np.argmax(test_logits, axis=1).flatten()

    output = []
    output.append(['Id', 'Predicted'])
    for i in range(len(result)):
        output.append([i, inttolabel(result[i])])

    with open('output_en.csv', 'w') as csv_file: 
        csv_w = csv.writer(csv_file, delimiter=",")
        for row in output: 
            csv_w.writerow(row)
        print("결과 저장 완료: ", datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S"))

      


결과 저장 완료:  2020/12/24 08:13:03
